In [ ]:
import torch
import numpy
import pandas
from discretize import generator, MarkovSampler

In [11]:
# number of stages:
T = 3
# number of assets:
N = 3
# risk free interest rate:
rf = 0.0005
# additional dataset:
fee = 0.001
coeffs = pandas.read_csv("./data/coefficients.csv",index_col=0)
beta = torch.tensor(coeffs['beta'])

/tmp/ipykernel_31164/2187737320.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = torch.tensor(coeffs['beta'])


In [13]:
# Markov chain discretization:
markovian = MarkovSampler(generator, n_Markov_states=[1] + [100] * (T - 1), n_samples=1000, T=T)
markovian.SA()
Markov_states = [None for _ in range(T)]
transition_matrix = markovian.transition_matrix
for t in range(T):
    # Markov_states (time, states, ((r_Mt, epsilon_Mt, sigma^2_Mt)))
    market_return = markovian.Markov_states[t][:,0].reshape(-1,1)
    asset_return_market_exposure = beta[:N]*(market_return-rf) + rf
    Markov_states[t] = torch.cat((asset_return_market_exposure,markovian.Markov_states[t]), axis=1)

In [ ]:
# https://github.com/lingquant/msppy/blob/master/doc/source/examples/portfolio_optimization/portfolio.ipynb

AssetMgt = MSLP(T=T, sense=-1, bound=200)